# Sequence Tagging With an RNN Using Poutyne

In this notebook, we will do sequence tagging with RNNs using Poutyne.

Let's install the latest version of Poutyne (if it is not already), and import all the needed packages.

In [1]:
%pip install --upgrade fasttext
%pip install --upgrade poutyne
%matplotlib inline
import contextlib
import os
import pickle
import re
import sys
from io import TextIOBase

import fasttext
import fasttext.util
import requests
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence, pad_sequence
from torch.utils.data import DataLoader

from poutyne import set_seeds, Model

Also, we need to set Python's, NumPy's and PyTorch's seeds by using Poutyne function so that our training is (almost) reproducible.

In [2]:
set_seeds(42)

# Train a Recurrent Neural Network (RNN)

We train an RNN, or more precisely, an LSTM, to predict the sequence of tags associated with a given address, which is also known as address parsing.

This task consists of detecting (by tagging) the different parts of an address such as the civic number, the street name or the postal code (or zip code). The following figure shows an example of such a tagging.

![address parsing canada](https://poutyne.org/_images/address_parsing.png)

Since addresses are written in a predetermined sequence, RNN is the best way to crack this problem. For our architecture, we will use two components, an RNN and a fully-connected layer.

## Training Constants

Now, let's set our training constants. We first have the CUDA device used for training if one is present. Secondly, we set the batch size (i.e. the number of elements to see before updating the model) and the learning rate for the optimizer.

In [3]:
cuda_device = 0
device = torch.device("cuda:%d" % cuda_device if torch.cuda.is_available() else "cpu")

batch_size = 32
lr = 0.1

## RNN

For the first components, instead of using a vanilla RNN, we use a variant of it, know as a long short-term memory (LSTM) (to learn more about [LSTM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/). For now, we use a single layer unidirectional LSTM. 

Also, since our data is textual, we will use the well-known word embeddings to encode the textual information. So the LSTM input and hidden state dimensions will be of the same size. This size corresponds to the word embeddings dimension, which in our case will be the [French pre trained](https://fasttext.cc/docs/en/crawl-vectors.html) fastText embeddings of dimension `300`.

> See [here](https://discuss.pytorch.org/t/could-someone-explain-batch-first-true-in-lstm/15402) the explanation why we use the `batch_first` argument.

In [4]:
dimension = 300
num_layer = 1
bidirectional = False

lstm_network = nn.LSTM(
    input_size=dimension,
    hidden_size=dimension,
    num_layers=num_layer,
    bidirectional=bidirectional,
    batch_first=True,
)

## Fully-connected Layer

We use this layer to map the representation of the LSTM (`300`) to the tag space (8, the number of tags) and predict the most likely tag using a softmax.

In [5]:
input_dim = dimension  # the output of the LSTM
tag_dimension = 8

fully_connected_network = nn.Linear(input_dim, tag_dimension)

## The Dataset

Now let's download our dataset; it already split into a train, valid and test set using the following.

In [6]:
def download_data(saving_dir, data_type):
    """
    Function to download the dataset using data_type to specify if we want the train, valid or test.
    """

    root_url = "https://graal-research.github.io/poutyne-external-assets/tips_and_tricks_assets/{}.p"

    url = root_url.format(data_type)
    r = requests.get(url)
    os.makedirs(saving_dir, exist_ok=True)

    open(os.path.join(saving_dir, f"{data_type}.p"), 'wb').write(r.content)


download_data('./datasets/addresses/', "train")
download_data('./datasets/addresses/', "valid")
download_data('./datasets/addresses/', "test")

Now let's load in memory the data.

In [7]:
# load the data

train_data = pickle.load(open("./datasets/addresses/train.p", "rb"))  # 80,000 examples
valid_data = pickle.load(open("./datasets/addresses/valid.p", "rb"))  # 20,000 examples
test_data = pickle.load(open("./datasets/addresses/test.p", "rb"))  # 30,000 examples

If we take a look at the training dataset, it's a list of `80,000` tuples where the first element is the full address, and the second element is a list of the tag (the ground truth).

In [8]:
train_data[0:2]

[('33 harnesworth crescent city of hamilton ontario l8b0j3',
  ['StreetNumber',
   'StreetName',
   'StreetName',
   'Municipality',
   'Municipality',
   'Municipality',
   'Province',
   'PostalCode']),
 ('1449 mouettes longueuil quebec j4j5k4',
  ['StreetNumber', 'StreetName', 'Municipality', 'Province', 'PostalCode'])]

Since the address is a text, we need to *convert* it into categorical value, such as word embeddings, for that we will use a vectorizer. This embedding vectorizer will be able to extract for every word embedding value.

In [9]:
# We use this class so that the download templating of the fasttext
# script be not buggy as hell in notebooks.
class LookForProgress(TextIOBase):
    def __init__(self, stdout):
        self.stdout = stdout
        self.regex = re.compile(r'([0-9]+(\.[0-9]+)?%)', re.IGNORECASE)

    def write(self, o):
        res = self.regex.findall(o)
        if len(res) != 0:
            print(f"\r{res[-1][0]}", end='', file=self.stdout)


class EmbeddingVectorizer:
    def __init__(self):
        """
        Embedding vectorizer
        """
        with contextlib.redirect_stdout(LookForProgress(sys.stdout)):
            fasttext.util.download_model('fr', if_exists='ignore')
        self.embedding_model = fasttext.load_model("./cc.fr.300.bin")

    def __call__(self, address):
        """
        Convert address to embedding vectors
        :param address: The address to convert
        :return: The embeddings vectors
        """
        embeddings = []
        for word in address.split():
            embeddings.append(self.embedding_model[word])
        return embeddings


embedding_model = EmbeddingVectorizer()

We also need a vectorizer to convert the address tag (e.g. StreetNumber, StreetName) into categorical values. So we will use a Vectorizer class that can use the embedding vectorizer and convert the address tag. We will explain and use the argument `predict` later.

In [10]:
class Vectorizer:
    def __init__(self, dataset, embedding_model, predict=False):
        self.data = dataset
        self.embedding_model = embedding_model
        self.predict = predict
        self.tags_set = {
            "StreetNumber": 0,
            "StreetName": 1,
            "Unit": 2,
            "Municipality": 3,
            "Province": 4,
            "PostalCode": 5,
            "Orientation": 6,
            "GeneralDelivery": 7,
        }

    def __len__(self):
        # for the dataloader
        return len(self.data)

    def __getitem__(self, item):
        data = self.data[item]

        if not self.predict:
            address = data[0]
            address_vector = self.embedding_model(address)

            tags = data[1]
            idx_tags = self._convert_tags_to_idx(tags)
            return address_vector, idx_tags

        address_vector = self.embedding_model(data)
        return address_vector

    def _convert_tags_to_idx(self, tags):
        idx_tags = []
        for tag in tags:
            idx_tags.append(self.tags_set[tag])
        return idx_tags

In [11]:
train_data_vectorized = Vectorizer(train_data, embedding_model)
valid_data_vectorized = Vectorizer(valid_data, embedding_model)
test_data_vectorized = Vectorizer(test_data, embedding_model)

### DataLoader

Now, since all the addresses are not of the same size, it is impossible to batch them together since all elements of a tensor must have the same lengths. But there is a trick, padding!

The idea is simple. We add *empty* tokens at the end of each sequence up to the longest one in a batch. For the word vectors, we add vectors of 0 as padding. For the tag indices, we pad with -100s. We do so because of the [cross-entropy loss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss), the accuracy metric and the [F1 metric](https://poutyne.org/metrics.html#poutyne.FBeta) all ignore targets with values of -100.

To do this padding, we use the `collate_fn` argument of the [PyTorch `DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader), and on running time, that process will be done. One thing to take into account, since we pad the sequence, we need each sequence's lengths to unpad them in the forward pass. That way, we can pad and pack the sequence to minimize the training time (read [this good explanation](https://stackoverflow.com/questions/51030782/why-do-we-pack-the-sequences-in-pytorch) of why we pad and pack sequences).

In [12]:
def pad_collate_fn(batch):
    """
    The collate_fn that can add padding to the sequences so all can have
    the same length as the longest one.

    Args:
        batch (List[List, List]): The batch data, where the first element
        of the tuple are the word idx and the second element are the target
        label.

    Returns:
        A tuple (x, y). The element x is a tuple containing (1) a tensor of padded
        word vectors and (2) their respective lengths of the sequences. The element
        y is a tensor of padded tag indices. The word vectors are padded with vectors
        of 0s and the tag indices are padded with -100s. Padding with -100 is done
        because the cross-entropy loss, the accuracy metric and the F1 metric ignores
        the targets with values -100.
    """

    # This gets us two lists of tensors and a list of integer.
    # Each tensor in the first list is a sequence of word vectors.
    # Each tensor in the second list is a sequence of tag indices.
    # The list of integer consist of the lengths of the sequences in order.
    sequences_vectors, sequences_labels, lengths = zip(
        *[
            (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
            for (seq_vectors, labels) in sorted(batch, key=lambda x: len(x[0]), reverse=True)
        ]
    )

    lengths = torch.LongTensor(lengths)

    padded_sequences_vectors = pad_sequence(sequences_vectors, batch_first=True, padding_value=0)

    padded_sequences_labels = pad_sequence(sequences_labels, batch_first=True, padding_value=-100)

    return (padded_sequences_vectors, lengths), padded_sequences_labels

In [13]:
train_loader = DataLoader(
    train_data_vectorized,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=pad_collate_fn,
    num_workers=2,
)
valid_loader = DataLoader(valid_data_vectorized, batch_size=batch_size, collate_fn=pad_collate_fn, num_workers=2)
test_loader = DataLoader(test_data_vectorized, batch_size=batch_size, collate_fn=pad_collate_fn, num_workers=2)

## Full Network

Since our sequences are of variable lengths and we want to be the most efficient possible by packing them, we cannot use the [PyTorch `nn.Sequential`](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) class to define our model, so we define the forward pass for it to pack and unpack the sequences (again, you can read [this good explanation](https://stackoverflow.com/questions/51030782/why-do-we-pack-the-sequences-in-pytorch) of why we pad and pack sequences).

In [14]:
class FullNetWork(nn.Module):
    def __init__(self, lstm_network, fully_connected_network):
        super().__init__()
        self.hidden_state = None

        self.lstm_network = lstm_network
        self.fully_connected_network = fully_connected_network

    def forward(self, padded_sequences_vectors, lengths):
        """
        Defines the computation performed at every call.
        """
        total_length = padded_sequences_vectors.shape[1]

        pack_padded_sequences_vectors = pack_padded_sequence(padded_sequences_vectors, lengths.cpu(), batch_first=True)

        lstm_out, self.hidden_state = self.lstm_network(pack_padded_sequences_vectors)
        lstm_out, _ = pad_packed_sequence(lstm_out, batch_first=True, total_length=total_length)

        tag_space = self.fully_connected_network(lstm_out)
        return tag_space.transpose(-1, 1)  # we need to transpose since it's a sequence


full_network = FullNetWork(lstm_network, fully_connected_network)

## Summary

So we have created an LSTM network (`lstm_network`), a fully connected network (`fully_connected_network`), those two components are used in the full network. This full network used padded, packed sequences (defined in the forward pass), so we created the `pad_collate_fn` function to process the needed work. The DataLoader will conduct that process. Finally, when we load the data, this will be done using the vectorizer, so the address will be represented using word embeddings. Also, the address components will be converted into categorical value (from 0 to 7).

# The Training Loop

Now that we have all the components for the network let's train our model.

In [15]:
optimizer = optim.SGD(full_network.parameters(), lr)
loss_function = nn.CrossEntropyLoss()

model = Model(
    full_network,
    optimizer,
    loss_function,
    batch_metrics=['accuracy'],
    device=device,
)

model.fit_generator(train_loader, valid_loader, epochs=10)

test_loss, test_acc = model.evaluate_generator(test_loader)

Epoch:  1/10 Train steps: 2500 Val steps: 625 17.43s loss: 0.402440 acc: 87.340841 val_loss: 0.074837 val_acc: 98.491375
Epoch:  2/10 Train steps: 2500 Val steps: 625 17.73s loss: 0.044888 acc: 99.044959 val_loss: 0.026150 val_acc: 99.355461
Epoch:  3/10 Train steps: 2500 Val steps: 625 17.63s loss: 0.019844 acc: 99.555795 val_loss: 0.015576 val_acc: 99.650330
Epoch:  4/10 Train steps: 2500 Val steps: 625 17.55s loss: 0.013548 acc: 99.688221 val_loss: 0.011891 val_acc: 99.731806
Epoch:  5/10 Train steps: 2500 Val steps: 625 17.63s loss: 0.010814 acc: 99.763319 val_loss: 0.009818 val_acc: 99.785883
Epoch:  6/10 Train steps: 2500 Val steps: 625 18.27s loss: 0.009196 acc: 99.790861 val_loss: 0.008578 val_acc: 99.789669
Epoch:  7/10 Train steps: 2500 Val steps: 625 17.93s loss: 0.008007 acc: 99.813415 val_loss: 0.007688 val_acc: 99.797362
Epoch:  8/10 Train steps: 2500 Val steps: 625 17.73s loss: 0.007109 acc: 99.833434 val_loss: 0.006799 val_acc: 99.834102
Epoch:  9/10 Train steps: 2500 V

# Predict on New Data

Now, let say we want to predict using our trained model. For these new addresses, we will only have the address and not the tags. Let us download this dataset.

In [16]:
download_data('./datasets/addresses/', "predict")
predict_data = pickle.load(open("./datasets/addresses/predict.p", "rb"))  # 30,000 examples

If we take a look at the predicted dataset, it is a list of 30,000 addresses.

In [17]:
predict_data[0:2]

['15 major st london ontario n5z1e1',
 '1032 king st w 1202 kingston ontario k7m9g2']

We also need to reuse the vectorizer, but now with the `predict` argument set to `True` since we cannot parse the ground truth.

In [18]:
predict_data_vectorized = Vectorizer(predict_data, embedding_model, predict=True)

We need to change a little the `pad_collate_fn` since we also pad the labels.

In [19]:
def pad_collate_fn_predict(batch):
    """
    The collate_fn add padding to the sequences so all can have
    the same length as the longest one.

    Args:
        batch (List[List]): The batch data of the word idx.

    Returns:
        A tuple (x, y). The element x is a tensor of padded word vectors, and y
        their respective lengths of the sequences. The word vectors are padded with vectors of 0s.
    """

    # This gets us two lists of tensors and a list of integers.
    # Each tensor in the first list is a sequence of word vectors.
    # The list of integers consists of the lengths of the sequences in order.

    sequences_vectors, lengths = zip(
        *[
            (torch.FloatTensor(seq_vectors), len(seq_vectors))
            for seq_vectors in sorted(batch, key=lambda x: len(x), reverse=True)
        ]
    )

    lengths = torch.LongTensor(lengths)

    padded_sequences_vectors = pad_sequence(sequences_vectors, batch_first=True, padding_value=0)

    return padded_sequences_vectors, lengths


predict_loader = DataLoader(
    predict_data_vectorized,
    batch_size=batch_size,
    collate_fn=pad_collate_fn_predict,
    num_workers=2,
)

Now, let's predict using the `predict_generator` method but without concatenating the returns since batches are not always the same size.

In [20]:
predictions = model.predict_generator(predict_loader, concatenate_returns=False)
predictions[0:2]

[array([[[ 10.362392  ,   3.4235473 ,  -2.209073  , ..., -12.281605  ,
          -14.648837  ,  -6.0009966 ],
         [  3.0805492 ,  11.114887  ,  13.809945  , ...,   4.4265227 ,
           -4.769767  , -11.348867  ],
         [  2.4337537 ,   0.28365183,   1.1139965 , ...,   2.737724  ,
           -0.5722164 ,   2.829398  ],
         ...,
         [ -2.9403193 , -10.5651045 , -13.855679  , ...,  -9.27598   ,
            6.3255806 ,  21.084066  ],
         [  1.1715801 ,   2.9763868 ,   3.6524632 , ...,  -0.72333026,
           -3.7734149 ,  -7.133936  ],
         [ -1.867637  ,  -3.8698483 ,  -5.643851  , ...,  -7.526398  ,
           -6.2222004 ,  -4.0403423 ]],
 
        [[ 11.595455  ,   3.7031417 ,  -2.3065462 , ..., -12.633009  ,
          -14.922238  ,  -6.347353  ],
         [  5.605796  ,  11.3620825 ,  13.836491  , ...,   3.9517322 ,
           -4.875977  , -11.297518  ],
         [  3.0809286 ,   1.3133283 ,   1.8499303 , ...,   3.0905602 ,
           -0.50734043,   2.7491

Or using the `predict_dataset` method:

In [21]:
predictions = model.predict_dataset(
    predict_data_vectorized,
    batch_size=batch_size,
    collate_fn=pad_collate_fn_predict,
    num_workers=2,
    concatenate_returns=False,
)

But, as you can see, we have log probabilities of our model, so we need to take the highest probability per address element.

In [22]:
idx_predictions = []
for batch in predictions:
    idx_predictions.extend(batch.argmax(axis=1).tolist())  # we take the highest argument (so the tag idx).

To convert those indexes into tags, we only need to convert them back using the inverse of the previous `tags_to_idx` dictionary.

In [23]:
idx_to_tags = {
    0: "StreetNumber",
    1: "StreetName",
    2: "Unit",
    3: "Municipality",
    4: "Province",
    5: "PostalCode",
    6: "Orientation",
    7: "GeneralDelivery",
}

tags_predictions = []
for address in idx_predictions:
    tags_predictions.append([idx_to_tags.get(tag) for tag in address])

tags_predictions[0:2]

[['StreetNumber',
  'StreetName',
  'StreetName',
  'StreetName',
  'Orientation',
  'Municipality',
  'Municipality',
  'Municipality',
  'Province',
  'PostalCode'],
 ['StreetNumber',
  'StreetName',
  'StreetName',
  'Orientation',
  'Unit',
  'Municipality',
  'Municipality',
  'Municipality',
  'Province',
  'PostalCode']]